In [1]:
%tensorflow_version 2.x
import tensorflow as tf
device_name = tf.test.gpu_device_name()
if device_name != '/device:GPU:0':
  raise SystemError('GPU device not found')
print('Found GPU at: {}'.format(device_name))

Found GPU at: /device:GPU:0


In [2]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
import datetime, os

In [3]:
# Clear any logs from previous runs
!rm -rf ./logs/ 


In [4]:
#importing pre-trained model
from tensorflow.python.keras.applications.vgg16 import VGG16
from tensorflow.python.keras.applications.resnet import ResNet50
from tensorflow.python.keras.applications.xception import Xception

#importing sequential model
from tensorflow.python.keras.models import Sequential
import matplotlib.pyplot as plt
import numpy as np

#importing layers
from tensorflow.python.keras.layers import Conv2D
from tensorflow.python.keras.layers import MaxPooling2D,GlobalAveragePooling2D
from tensorflow.python.keras.layers import Flatten,Dropout
from tensorflow.python.keras.layers import Dense,GaussianNoise
from tensorflow.python.keras.models import load_model

from tensorflow.python.keras.callbacks import History 
from sklearn.utils import class_weight
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [6]:
cd '/content/drive/My Drive/Project Deep Learning/Segmented'

/content/drive/My Drive/Project Deep Learning/Segmented


In [7]:
def create_model():
  classifier = Sequential()
  resnet  = ResNet50(include_top = False,weights = 'imagenet',input_shape = (224,224,3))


  #adding layers
  classifier.add(resnet)
  classifier.add(MaxPooling2D())
  classifier.add(Flatten())
  classifier.add(Dense(units=512,activation='relu'))
  classifier.add(GaussianNoise(0.1))
  classifier.add(Dense(units=512,activation='relu'))
  classifier.add(GaussianNoise(0.1))
  classifier.add(Dense(units = 7,activation = 'softmax'))

  return classifier


In [8]:

def load_data():
  #Generating Dataset
  from keras.preprocessing.image import ImageDataGenerator

  train_datagen = ImageDataGenerator(rescale=1./255,
      shear_range=0.2,rotation_range=90,
      zoom_range=0.2,
      horizontal_flip=True,
      validation_split=0.28) # set validation split

  train_generator = train_datagen.flow_from_directory('/content/drive/My Drive/Project Deep Learning/Segmented',
                                                      target_size = (224,224),
                                                      color_mode = 'rgb',
                                                      class_mode = 'categorical',
                                                      classes =  ['Rust','Powdery_mildew','Mosaic','Downy_mildew','Copper_Phytotoxicity','Brown_Spot','Bacterial_blight'],
                                                      batch_size = 32 , 
                                                      subset='training') # set as training dat

  validation_generator = train_datagen.flow_from_directory('/content/drive/My Drive/Project Deep Learning/Segmented',
                                                            target_size = (224,224),
                                                            color_mode = 'rgb',
                                                            shuffle = False,
                                                            class_mode = 'categorical',
                                                            classes = ['Rust','Powdery_mildew','Mosaic','Downy_mildew','Copper_Phytotoxicity','Brown_Spot','Bacterial_blight'],
                                                            batch_size =32 ,
                                                            subset='validation') # set as validation data
  return train_generator,validation_generator



In [ ]:
def train_model(train_generator, validation_generator, history):
  classifier = create_model()
  #compiling model
  classifier.compile(optimizer = 'sgd',metrics = ['accuracy'],loss = 
  'categorical_crossentropy')

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

  batch_size=32
  classifier.fit(
      train_generator,
      steps_per_epoch =237,
      validation_data = validation_generator, 
      validation_steps =87,
      epochs =20,
      workers=8,
      callbacks=[history,tensorboard_callback])
  classifier.save("minor_project_model")

In [ ]:
%tensorboard --logdir logs

In [9]:
def plot_graph(history):
  # Plot training & validation accuracy values
  plt.plot(history.history['accuracy'])
  plt.plot(history.history['val_accuracy'])
  plt.title('Model accuracy')
  plt.ylabel('Accuracy')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

  # Plot training & validation loss values
  plt.plot(history.history['loss'])
  plt.plot(history.history['val_loss'])
  plt.title('Model loss')
  plt.ylabel('Loss')
  plt.xlabel('Epoch')
  plt.legend(['Train', 'Test'], loc='upper left')
  plt.show()

In [10]:
def predict():
  classifier = tf.keras.models.load_model("minor_project_model")
  train_generator, validation_generator = load_data()
  pred = classifier.predict(validation_generator,steps=len(validation_generator))
  predict_classes = list(np.argmax(pred,axis=1))
  true_classes = validation_generator.classes
  classifier.summary()
  return predict_classes, true_classes

In [11]:
def get_classwise_accuracy():
  
  predict_classes, y_true = predict()
  cm = confusion_matrix(y_true, predict_classes)
  classes = ['Rust','Powdery_mildew','Mosaic','Downy_mildew','Copper_Phytotoxicity','Brown_Spot','Bacterial_blight']
  total = np.sum(cm,axis=1)
  accuracy=[]
  for i in range(cm.shape[0]):
    accuracy.append(cm[i][i]/total[i])
  
  return cm, accuracy


In [ ]:
train_generator, validation_generator = load_data()
history = History()
train_model(train_generator, validation_generator, history)

In [ ]:
plot_graph(history)

In [12]:
confusion_matrix, accuracy = get_classwise_accuracy()

Using TensorFlow backend.


Found 213 images belonging to 7 classes.
Found 79 images belonging to 7 classes.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resnet50 (Model)             (None, 7, 7, 2048)        23587712  
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 3, 3, 2048)        0         
_________________________________________________________________
flatten (Flatten)            (None, 18432)             0         
_________________________________________________________________
dense (Dense)                (None, 512)               9437696   
_________________________________________________________________
gaussian_noise (GaussianNois (None, 512)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 512)               262656    
_________________________________________

In [13]:
confusion_matrix

array([[17,  1,  0,  0,  0,  0,  0],
       [ 0, 21,  0,  0,  0,  0,  0],
       [ 0,  0,  6,  0,  0,  0,  0],
       [ 0,  1,  0,  8,  0,  0,  0],
       [ 0,  0,  0,  0,  2,  1,  3],
       [ 1,  0,  1,  0,  0,  2,  1],
       [ 0,  0,  1,  0,  0,  0, 13]])

In [14]:
accuracy

[0.9444444444444444,
 1.0,
 1.0,
 0.8888888888888888,
 0.3333333333333333,
 0.4,
 0.9285714285714286]